In [ ]:
%matplotlib widget
import h5py
import numpy as np
from matplotlib import pyplot as plt
#To import DanMAX from the folder above:
import sys
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
fname = DM.findScan().replace('.h5', '_dxchange.h5')

In [ ]:
def calc_com(binary_image):
    hor = np.sum(binary_image, axis=0)
    ver = np.sum(binary_image, axis=1)
    com_x = 0
    com_y = 0
    for i in range(0, len(hor)):
        com_x += hor[i]*i
    for i in range(0, len(ver)):
        com_y += ver[i]*i
    com_x = com_x/np.sum(hor)
    com_y = com_y/np.sum(ver)
    return com_x, com_y

f = h5py.File(fname, 'r')
dark = np.mean(f['exchange/data_dark'][()], axis = 0)
flat = np.mean(f['exchange/data_white'][()], axis = 0)
images = f['exchange/data'][()]
pixel_size = f['/instrument/detector/actual_pixel_size_x'][()]*1e-3

In [ ]:
images = (images - dark)/(flat - dark)

In [ ]:
# initialize figure
res = DM.interactiveImageHist(images[0],ignore_zero=True)

In [ ]:
threshold = 0.7

In [ ]:
images = (images < threshold)*1

In [ ]:
x = []
y = []
for i in images:
    com_x, com_y = calc_com(i)
    x.append(com_x)
    y.append(com_y)

In [ ]:
print('Y variation is {:.2f} px'.format(np.max(y)-np.min(y)))
print('X variation is {:.2f} px'.format(np.max(x)-np.min(x)))

In [ ]:
plt.figure()
plt.plot(x, y, marker = 'X')
plt.show

In [ ]:
cam_rot = np.rad2deg(np.arcsin(np.polyfit(x, y, 1)[0]))
print('camera rotation is {} deg'.format(cam_rot))
print('suggested motion is')
print('umvr nf_micro_cam_rot {:.4f}'.format(-cam_rot))

In [ ]:
tilt = np.rad2deg(np.arctan((np.max(y)-np.min(y))/(np.max(x)-np.min(x))))
print('im_rx tilt is {} deg'.format(tilt))
print('suggested motion is')
print('umvr tom_rx {:.4f}'.format(tilt))

In [ ]:
cor_offset =  (images.shape[2]/2) - (0.5*(np.max(x)+np.min(x)))
print('rotation axis offset is {} pixels ({} mm) relative to the image center'.format(cor_offset, cor_offset*pixel_size))
print('suggested motion is')
print('umvr tom_x {:.4f}'.format(cor_offset*pixel_size))